In [5]:
import numpy as np
import pandas as pd

In [6]:
file = "../playground/ibamco_sheet.xlsx"
zip_file = "../data/zip/zips.xlsx"
zip1 = "main"
zip2 = "next"
sheet = "main"

In [7]:
import re
def clean_string(s):
    """remove all non-alphanumeric characters from a string, convert spanish characters to english"""
    s = s.lower()
    s = s.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace("ñ", "n").replace("ü", "u").replace("ç", "c").replace("à", "a").replace("è", "e").replace("ì", "i").replace("ò", "o").replace("ù", "u").replace("ä", "a").replace("ë", "e").replace("ï", "i").replace("ö", "o").replace("ü", "u").replace("â", "a").replace("ê", "e").replace("î", "i").replace("ô", "o").replace("û", "u").replace("ã", "a").replace("õ", "o").replace("ñ", "n").replace("ç", "c").replace("ß", "ss").replace("æ", "ae").replace("œ", "oe").replace('/', ' ').replace('_', ' ')
    # dont remove spaces or caps
    s = re.sub(r'[^a-zA-Z0-9 ]', '', s)
    return s

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def zf_idf_scores(text1, text2):
    tfidf = TfidfVectorizer().fit_transform([text1, text2])
    return cosine_similarity(tfidf)[0,1]

In [15]:
def normal_search(search, items):
    for idx, item in enumerate(items):
        if search == item:
            return idx
    return None

def idx_match(search, items, quick=False):
    if quick:
        id = normal_search(search, items)
        if id is not None:
            return id
    scores = [zf_idf_scores(search, item) for item in items]
    idx = np.argmax(scores)
    if scores[idx] < 0.5:
        return None
    if scores[idx] < 0.8:
        print(search, items[idx])
    return idx

In [16]:
df = pd.read_excel(file, sheet)
zips = pd.read_excel(zip_file, zip1).sort_values(by='poblacion').drop_duplicates()
zips.columns = [clean_string(c) for c in zips.columns]
zips['prob_c'] = zips['poblacion'].apply(lambda x: clean_string(x))
zips['comarca2'] = None
zips['provincia2'] = None
zips['comunidad2'] = None
zips['code2'] = None
zips['prob_2'] = None

prov = pd.read_excel(zip_file, zip2).sort_values(by='municipio').drop_duplicates()
prov.columns = [clean_string(c) for c in prov.columns]
prov['prob_m'] = prov['municipio'].apply(lambda x: clean_string(x))

In [17]:
zips

,codigo postal,lat,lon,poblacion,provincia,comunidad autonoma,prob_c,comarca2,provincia2,comunidad2,code2,prob_2
0,25651,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,None,None,None,None,None
1,25652,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,None,None,None,None,None
2,25747,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,None,None,None,None,None
3,8630,1.902210e+09,41517943520,Abrera,Barcelona,Cataluña,abrera,None,None,None,None,None
4,46140,-1.286690e+09,40061200130,Ademuz,Valencia/València,Comunidad Valenciana,ademuz,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
8,25692,7.628481e-01,41999559420,Àger,Lleida,Cataluña,ager,None,None,None,None,None
7,25691,7.628481e-01,41999559420,Àger,Lleida,Cataluña,ager,None,None,None,None,None
6,25611,7.628481e-01,41999559420,Àger,Lleida,Cataluña,ager,None,None,None,None,None
1392,8711,1.638468e+09,41605410910,Òdena,Barcelona,Cataluña,odena,None,None,None,None,None


In [18]:
prov

,municipio,codigo ine,comarca,provincia,comunidad autonoma,prob_m
47,Abella de la Conca,250019.0,Pallars Jussà,Lerida/ Lleida,Cataluña/ Catalunya,abella de la conca
48,Abrera,80018.0,Baix Llobregat,Barcelona,Cataluña/ Catalunya,abrera
49,Ademuz,46001.0,El Rincón de Ademuz,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,ademuz
50,Ador,46002.0,La Safor,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,ador
52,Agost,3002.0,L'Alacantí,Alicante/Alacant,Comunidad Valenciana/ Comunitat valenciana,agost
...,...,...,...,...,...,...
46,Sóller y Valldemosa,NaN,La Sierra de Tramontana/ sa Serra de Tramuntana,Mallorca,Islas Baleares/ Illes balears,solleryvalldemosa
51,Àger,250024.0,Noguera,Lerida/ Lleida,Cataluña/ Catalunya,ager
548,"Énova, l'",46119.0,La Ribera Alta,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,enova l
882,Òdena,81430.0,Anoia,Barcelona,Cataluña/ Catalunya,odena


In [19]:
from tqdm import tqdm
from time import sleep

In [20]:
prov.sort_index(inplace=True)
zips.sort_index(inplace=True)
prov.index = range(len(prov))
zips.index = range(len(zips))

In [21]:
for idx, row in tqdm(zips.iterrows(), total=zips.shape[0]):
    mn = row['prob_c']
    prov_idx = idx_match(mn, prov['prob_m'].values, quick=True)
    if prov_idx is not None:
        zips.loc[idx, 'comarca2'] = prov.loc[prov_idx, 'comarca']
        zips.loc[idx, 'provincia2'] = prov.loc[prov_idx, 'provincia']
        zips.loc[idx, 'comunidad2'] = prov.loc[prov_idx, 'comunidad autonoma']
        zips.loc[idx, 'code2'] = prov.loc[prov_idx, 'codigo ine']
        zips.loc[idx, 'prob_2'] = prov.loc[prov_idx, 'prob_m']
    

  0%|          | 0/2364 [00:00<?, ?it/s]

  4%|▍         | 101/2364 [00:19<08:30,  4.43it/s]

algimia de alfara alfara de carles


  5%|▌         | 119/2364 [01:37<2:46:39,  4.45s/it]

In [17]:
zips['codigo postal'] = zips['codigo postal'].astype(str).apply(lambda x: x.zfill(5))
zips['code2'].fillna('', inplace=True)
zips['code2'] = zips['code2'].apply(lambda x: str(int(float(x))).zfill(5) if ('nan' != x and 'None' !=  x and not pd.isna(x) and x != '') else x)
zips

,codigo postal,lat,lon,poblacion,provincia,comunidad autonoma,prob_c,comarca2,provincia2,comunidad2,code2,prob_2
0,25651,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,Pallars Jussà,Lerida/ Lleida,Cataluña/ Catalunya,250019,abella de la conca
1,25652,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,Pallars Jussà,Lerida/ Lleida,Cataluña/ Catalunya,250019,abella de la conca
2,25747,1.091772e+09,42161421170,Abella de la Conca,Lleida,Cataluña,abella de la conca,Pallars Jussà,Lerida/ Lleida,Cataluña/ Catalunya,250019,abella de la conca
3,08630,1.902210e+09,41517943520,Abrera,Barcelona,Cataluña,abrera,Baix Llobregat,Barcelona,Cataluña/ Catalunya,80018,abrera
4,46140,-1.286690e+09,40061200130,Ademuz,Valencia/València,Comunidad Valenciana,ademuz,El Rincón de Ademuz,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,46001,ademuz
...,...,...,...,...,...,...,...,...,...,...,...,...
2359,46950,-4.252401e-01,39465751320,Xirivella,Valencia/València,Comunidad Valenciana,xirivella,L'Horta Sud,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,46110,xirivella
2360,46367,-8.082808e-01,39384514380,Yátova,Valencia/València,Comunidad Valenciana,yatova,La Hoya de Buñol,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,46261,yatova
2361,46621,-1.074753e+09,39091525630,Zarra,Valencia/València,Comunidad Valenciana,zarra,El Valle de Cofrentes-Ayora,Valencia/València,Comunidad Valenciana/ Comunitat valenciana,46263,zarra
2362,12311,-1.659027e-01,40728506150,Zorita del Maestrazgo,Castellón/Castelló,Comunidad Valenciana,zorita del maestrazgo,Els Ports,Castellón/Castelló,Comunidad Valenciana/ Comunitat valenciana,12141,zorita del maestrazgo


In [152]:
# zips['code2'].apply(lambda x: str(int(float(x))).zfill(5) if ('nan' not in x and 'None' not in x) else x)

TypeError: argument of type 'numpy.float64' is not iterable

In [18]:
zips.to_excel("../data/zip/zips3.xlsx", 'main', index=False, )

In [58]:
prov[prov.prob_m.str.contains('Agost')]

,municipio,codigo INE,comarca,provincia,Comunidad Autonoma,prob_m
52,Agost,3002.0,L'Alacantí,Alicante/Alacant,Comunidad Valenciana/ Comunitat valenciana,Agost


In [20]:
zips[zips.comarca2.isna()]

,codigo postal,lat,lon,poblacion,provincia,comunidad autonoma,prob_c,comarca2,provincia2,comunidad2,code2,prob_2
65,03841,-4.019704e-01,38794472240,Alcocer de Planes,Alicante/Alacant,Comunidad Valenciana,alcocer de planes,None,None,None,,None
87,46197,-5.595106e-01,39276809660,Alfarp,Valencia/València,Comunidad Valenciana,alfarp,None,None,None,,None
171,07150,2.421667e+09,39576985670,Andratx,Illes Balears,Illes Balears,andratx,None,None,None,,None
172,07157,2.421667e+09,39576985670,Andratx,Illes Balears,Illes Balears,andratx,None,None,None,,None
173,07159,2.421667e+09,39576985670,Andratx,Illes Balears,Illes Balears,andratx,None,None,None,,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1988,07101,2.714138e+09,39764457300,Sóller,Illes Balears,Illes Balears,soller,None,None,None,,None
1989,07108,2.714138e+09,39764457300,Sóller,Illes Balears,Illes Balears,soller,None,None,None,,None
1990,07179,2.714138e+09,39764457300,Sóller,Illes Balears,Illes Balears,soller,None,None,None,,None
2221,07170,2.622184e+09,39709269210,Valldemossa,Illes Balears,Illes Balears,valldemossa,None,None,None,,None


In [ ]:
np.argmax